### NOTES:


In [ ]:
# Import dependencies
from pprint import pprint
import pandas as pd 
import requests
import numpy as np

from config import yelp_api_key

In [ ]:
# Generate offsets list to loop through more than 50 results up to 1000
offsets = np.arange(0,1000,50)
offsets

In [ ]:
# For testing
# offsets = np.arange(0,50,50)

In [ ]:
# Loop through Chicago zip codes 
# List of Chicago zip codes comes from https://zipcode.org/city/IL/CHICAGO
chi_zips = ['60701','60696','60693','60691','60690','60685','60684','60699',
            '60697','60695','60694','60689','60688','60687','60686','60682',
            '60681','60680','60678','60675','60674','60670','60669','60677',
            '60673','60668','60666','60659','60657','60655','60654','60651',
            '60649','60645','60644','60640','60639','60664','60661','60660',
            '60656','60653','60652','60647','60646','60643','60642','60641',
            '60638','60634','60633','60631','60637','60636','60632','60630',
            '60629','60626','60625','60621','60620','60617','60616','60613',
            '60612','60609','60628','60624','60623','60622','60619','60618',
            '60615','60614','60611','60610','60608','60607','60606','60605',
            '60604','60603','60602','60601','60290']

In [ ]:
# For testing
# chi_zips = ["60622"]

In [ ]:
# Set empty list to store total search results per zip code
totals = []

In [ ]:
# Set empty lists to store restaurant info
name = []
category = []
category_title = []
rating = []
price = []
review_count = []
display_address = []
address1 = []
address2 = []
address3 = []
city = []
latitude = []
longitude = []
zip_code = []
restaurant_id = []

In [ ]:
# Yelp Fusion API info here: https://www.yelp.com/developers/documentation/v3/business_search
base_url = "https://api.yelp.com/v3/businesses/search"

# Loop through Chicago zip codes list
for zips in chi_zips:
    
    location = f'Chicago, {zips}'
    
    # Loop through offsets to get each group of 50 results
    for offset in offsets:
        params = {'term' : 'restaurant',
                  'location' : location,
                  'limit' : 50,
                  'offset' : offset}

        # Headers contain the API key
        headers = {'Authorization': 'Bearer {}'.format(yelp_api_key)}

        # Place request
        response = requests.get(base_url, headers=headers, params=params)

        # JSON-ify data
        data = response.json()

        if 'businesses' in data.keys():
            
        # Loop through number of businesses and append values to lists
            for i in range(len(data['businesses'])):

                # Append values to lists
                try:
                    name.append(data['businesses'][i]['name'])
                except KeyError:
                    name.append(np.nan)

                try:
                    rating.append(data['businesses'][i]['rating'])
                except KeyError:
                    rating.append(np.nan)

                try:
                    price.append(data['businesses'][i]['price'])
                except KeyError:
                    price.append(np.nan)

                try:
                    review_count.append(data['businesses'][i]['review_count'])
                except KeyError:
                    review_count.append(np.nan)

                try:
                    display_address.append(data['businesses'][i]['location']['display_address'])
                except KeyError:
                    display_address.append(np.nan)
                
                try:
                    address1.append(data['businesses'][i]['location']['address1'])
                except KeyError:
                    address1.append(np.nan)
                    
                try:
                    address2.append(data['businesses'][i]['location']['address2'])
                except KeyError:
                    address2.append(np.nan)
                    
                try:
                    address3.append(data['businesses'][i]['location']['address3'])
                except KeyError:
                    address3.append(np.nan)
                    
                try:
                    city.append(data['businesses'][i]['location']['city'])
                except KeyError:
                    city.append(np.nan)
                    
                try:
                    latitude.append(data['businesses'][i]['coordinates']['latitude'])
                except KeyError:
                    latitude.append(np.nan)

                try:
                    longitude.append(data['businesses'][i]['coordinates']['longitude'])
                except KeyError:
                    longitude.append(np.nan)

                try:
                    zip_code.append(data['businesses'][i]['location']['zip_code'])
                except KeyError:
                    zip_code.append(np.nan)

                try:
                    restaurant_id.append(data['businesses'][i]['id'])
                except:
                    restaurant_id.append(np.nan)

                try:
                    category.append(data['businesses'][i]['categories'][0]['alias'])
                except:
                    category.append(np.nan)

                try:
                    category_title.append(data['businesses'][i]['categories'][0]['title'])
                except:
                    category_title.append(np.nan)
        
        totals.append(data['total'])
    

In [ ]:
# Pretty print
# pprint(data)

In [ ]:
# Stores number of results per zip code
totals = pd.Series(totals).unique()
totals

In [ ]:
# Build datadrame from lists 
restaurants_df = pd.DataFrame({'name' : name,
                              'category' : category,
                              'category_title' : category_title, 
                              'rating' : rating,
                              'price' : price,
                              'review_count' : review_count,
                              'display_address' : display_address,
                              'address1' : address1,
                              'address2' : address2,
                              'address3' : address3,
                              'city' : city, 
                              'latitude' : latitude,
                              'longitude' : longitude,
                              'zip code' : zip_code,
                              'restaurant_id' : restaurant_id})

In [ ]:
# View dataframe
restaurants_df

In [ ]:
# Check the number of unique restaurant IDs
len(restaurants_df['restaurant_id'].unique())

In [ ]:
restaurants_df.count()

In [ ]:
# Inspect data to see how many zip codes we have in the existing dataframe
restaurants_df['zip code'].value_counts()

In [ ]:
# Export to .csv
# restaurants_df.to_csv('yelp_csv/yelp_csv.csv')